In [ ]:
# Run this cell to install DiffeRT and its dependencies, e.g., on Google Colab

try:
    import differt  # noqa: F401
except ImportError:
    import sys  # noqa: F401

    !{sys.executable} -m pip install differt[all]

# Coherence Map

In [ ]:
import equinox as eqx
import jax.numpy as jnp

from differt.scene.sionna import download_sionna_scenes, get_sionna_scene
from differt.scene.triangle_scene import TriangleScene

In [ ]:
download_sionna_scenes()  # Let's download Sionna scenes (from the main branch)

In [ ]:
file = get_sionna_scene("simple_street_canyon")
scene = TriangleScene.load_xml(file)
scene = eqx.tree_at(lambda s: s.transmitters, scene, jnp.array([-33, 11, 32.0]))
scene = eqx.tree_at(lambda s: s.receivers, scene, jnp.array([-15, 0, 1.5]))
scene.plot()

In [ ]:
paths = scene.compute_paths(order=0)
canvas = scene.plot()
paths.plot(canvas=canvas)